In [6]:
"""
Opening Range Breakout (ORB) Strategy - Based on Video Method
===============================================================
1. Mark first 15-min candle (9:30-9:45 AM) high/low
2. Wait for 5-min breakout with Fair Value Gap overlapping the range
3. Wait for retest of range high/low
4. Enter on engulfing candle confirmation
5. Stop: swing low/high | Target: 2R
"""
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import pytz

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"
CONTRACT_ID = "CON.F.US.MNQ.H26"

# Strategy parameters
POSITION_SIZE = 20
RISK_REWARD_RATIO = 2  # Target is 2x the stop loss distance

# Realism parameters
SLIPPAGE_POINTS = 0.25  # 1 tick for MNQ
COMMISSION_PER_CONTRACT = 0.62  # Per side
POINT_VALUE = 0.5  # $0.50 per point for MNQ

# ORB parameters
ORB_START_TIME = (9, 30)  # 9:30 AM
ORB_END_TIME = (9, 45)    # 9:45 AM (first 15-min candle)

# Trade management
MAX_TRADES_PER_DAY = 1

# ==================== API FUNCTIONS ====================
def authenticate():
    resp = requests.post(
        f"{API_BASE}/Auth/loginKey",
        json={"userName": USERNAME, "apiKey": KEY},
        headers={"accept": "text/plain", "Content-Type": "application/json"},
        timeout=15
    )
    return {
        "Authorization": f"Bearer {resp.json()['token']}",
        "accept": "text/plain",
        "Content-Type": "application/json"
    }

def fetch_bars(headers, unit, unit_number):
    end_time = datetime.now(pytz.UTC)
    start_time = end_time - timedelta(days=30)  # Fetch 30 days of data
    
    payload = {
        "contractId": CONTRACT_ID,
        "live": False,
        "startTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "endTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "unit": unit,
        "unitNumber": unit_number,
        "limit": 10000,
        "includePartialBar": True
    }
    
    resp = requests.post(f"{API_BASE}/History/retrieveBars", json=payload, headers=headers, timeout=30)
    return resp.json().get("bars", [])

def parse_bars_with_ny_time(bars):
    ny_tz = pytz.timezone('America/New_York')
    parsed = []
    
    for bar in bars:
        timestamp = bar['t'].replace('+00:00', 'Z')
        dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        dt_utc = pytz.utc.localize(dt_utc)
        dt_ny = dt_utc.astimezone(ny_tz)
        
        bar['dt_ny'] = dt_ny
        bar['date'] = dt_ny.date()
        parsed.append(bar)
    
    return parsed

# ==================== ORB FUNCTIONS ====================
def get_orb_range(bars_15m, today, verbose=True):
    """Get the 9:30-9:45 AM opening range"""
    if verbose:
        print(f"\n{'='*70}")
        print(f"🔍 STEP 1: Looking for 9:30-9:45 AM opening range...")
        print(f"{'='*70}")
    
    for bar in bars_15m:
        if bar['date'] == today:
            if bar['dt_ny'].hour == 9 and bar['dt_ny'].minute == 30:
                orb_high = bar['h']
                orb_low = bar['l']
                orb_open = bar['o']
                orb_close = bar['c']
                orb_size = orb_high - orb_low
                
                if verbose:
                    print(f"\n✅ ORB Range Found (9:30-9:45 AM):")
                    print(f"   Open:  {orb_open:.2f}")
                    print(f"   High:  {orb_high:.2f}")
                    print(f"   Low:   {orb_low:.2f}")
                    print(f"   Close: {orb_close:.2f}")
                    print(f"   Size:  {orb_size:.2f} points")
                
                return {
                    'high': orb_high,
                    'low': orb_low,
                    'open': orb_open,
                    'close': orb_close,
                    'size': orb_size
                }
    
    if verbose:
        print(f"\n❌ No ORB range found for {today}")
    return None

def has_fair_value_gap(bar1, bar2, bar3, verbose=False):
    """
    Check if 3 consecutive bars form a Fair Value Gap
    FVG = low of 3rd candle doesn't touch high of 1st candle (bullish)
         OR high of 3rd candle doesn't touch low of 1st candle (bearish)
    """
    if verbose:
        print(f"\n      🔬 Checking FVG:")
        print(f"         Bar 1: H={bar1['h']:.2f} L={bar1['l']:.2f}")
        print(f"         Bar 2: H={bar2['h']:.2f} L={bar2['l']:.2f} (breakout)")
        print(f"         Bar 3: H={bar3['h']:.2f} L={bar3['l']:.2f}")
    
    # Bullish FVG: gap between bar1 high and bar3 low
    bullish_gap = bar3['l'] > bar1['h']
    
    # Bearish FVG: gap between bar1 low and bar3 high
    bearish_gap = bar3['h'] < bar1['l']
    
    if bullish_gap:
        if verbose:
            print(f"         ✅ BULLISH FVG: Bar3 Low ({bar3['l']:.2f}) > Bar1 High ({bar1['h']:.2f})")
            print(f"         Gap range: {bar1['h']:.2f} to {bar3['l']:.2f}")
        return 'BULLISH', (bar1['h'], bar3['l'])
    elif bearish_gap:
        if verbose:
            print(f"         ✅ BEARISH FVG: Bar3 High ({bar3['h']:.2f}) < Bar1 Low ({bar1['l']:.2f})")
            print(f"         Gap range: {bar3['h']:.2f} to {bar1['l']:.2f}")
        return 'BEARISH', (bar3['h'], bar1['l'])
    
    if verbose:
        print(f"         ❌ No FVG found")
    
    return None, None

def find_breakout_with_fvg(bars_5m, orb, today, verbose=True):
    """
    Find 5-min breakout with Fair Value Gap that overlaps the ORB range
    """
    if verbose:
        print(f"\n🔍 STEP 2: Looking for 5-min breakout with FVG...")
        print(f"   ORB High: {orb['high']:.2f}")
        print(f"   ORB Low:  {orb['low']:.2f}")
    
    # Get bars after 9:45 AM
    bars_after_orb = []
    for bar in bars_5m:
        if bar['date'] == today:
            bar_time = bar['dt_ny'].hour * 60 + bar['dt_ny'].minute
            if bar_time >= (9 * 60 + 45):  # After 9:45 AM
                bars_after_orb.append(bar)
    
    # Sort chronologically
    bars_after_orb.sort(key=lambda x: x['dt_ny'])
    
    if verbose:
        print(f"\n   Scanning {len(bars_after_orb)} 5-min bars after 9:45 AM...")
    
    # Check each set of 3 consecutive bars for breakout + FVG
    for i in range(len(bars_after_orb) - 2):
        bar1 = bars_after_orb[i]
        bar2 = bars_after_orb[i + 1]  # Middle candle (breakout candle)
        bar3 = bars_after_orb[i + 2]
        
        if verbose:
            print(f"\n   ⏰ {bar2['dt_ny'].strftime('%I:%M %p')} - Checking bar...")
            print(f"      O={bar2['o']:.2f} H={bar2['h']:.2f} L={bar2['l']:.2f} C={bar2['c']:.2f}")
        
        # Check if middle candle breaks out
        broke_high = bar2['c'] > orb['high']
        broke_low = bar2['c'] < orb['low']
        
        if broke_high:
            if verbose:
                print(f"      📈 Closed ABOVE ORB high ({bar2['c']:.2f} > {orb['high']:.2f})")
            
            # Check for FVG
            fvg_type, fvg_range = has_fair_value_gap(bar1, bar2, bar3, verbose=verbose)
            
            if fvg_type == 'BULLISH':
                # Check if FVG overlaps with ORB high
                fvg_overlaps = fvg_range[0] <= orb['high'] <= fvg_range[1]
                
                if verbose:
                    print(f"\n      🔍 Checking FVG overlap with ORB high:")
                    print(f"         FVG range: {fvg_range[0]:.2f} - {fvg_range[1]:.2f}")
                    print(f"         ORB high: {orb['high']:.2f}")
                    print(f"         Overlaps: {fvg_overlaps}")
                
                if fvg_overlaps:
                    if verbose:
                        print(f"\n   ✅✅✅ VALID LONG BREAKOUT WITH FVG ✅✅✅")
                        print(f"   Time: {bar2['dt_ny'].strftime('%I:%M %p')}")
                        print(f"   Breakout close: {bar2['c']:.2f}")
                        print(f"   FVG range: {fvg_range[0]:.2f} - {fvg_range[1]:.2f}")
                    
                    return {
                        'direction': 'LONG',
                        'breakout_bar': bar2,
                        'fvg_range': fvg_range,
                        'fvg_high': fvg_range[1],
                        'fvg_low': fvg_range[0],
                        'bar1': bar1,
                        'bar3': bar3
                    }
                elif verbose:
                    print(f"      ❌ FVG doesn't overlap ORB high - skipping")
        
        elif broke_low:
            if verbose:
                print(f"      📉 Closed BELOW ORB low ({bar2['c']:.2f} < {orb['low']:.2f})")
            
            # Check for FVG
            fvg_type, fvg_range = has_fair_value_gap(bar1, bar2, bar3, verbose=verbose)
            
            if fvg_type == 'BEARISH':
                # Check if FVG overlaps with ORB low
                fvg_overlaps = fvg_range[0] <= orb['low'] <= fvg_range[1]
                
                if verbose:
                    print(f"\n      🔍 Checking FVG overlap with ORB low:")
                    print(f"         FVG range: {fvg_range[0]:.2f} - {fvg_range[1]:.2f}")
                    print(f"         ORB low: {orb['low']:.2f}")
                    print(f"         Overlaps: {fvg_overlaps}")
                
                if fvg_overlaps:
                    if verbose:
                        print(f"\n   ✅✅✅ VALID SHORT BREAKOUT WITH FVG ✅✅✅")
                        print(f"   Time: {bar2['dt_ny'].strftime('%I:%M %p')}")
                        print(f"   Breakout close: {bar2['c']:.2f}")
                        print(f"   FVG range: {fvg_range[0]:.2f} - {fvg_range[1]:.2f}")
                    
                    return {
                        'direction': 'SHORT',
                        'breakout_bar': bar2,
                        'fvg_range': fvg_range,
                        'fvg_high': fvg_range[1],
                        'fvg_low': fvg_range[0],
                        'bar1': bar1,
                        'bar3': bar3
                    }
                elif verbose:
                    print(f"      ❌ FVG doesn't overlap ORB low - skipping")
        elif verbose:
            print(f"      ⏸️  Still inside ORB range")
    
    if verbose:
        print(f"\n   ❌ No valid breakout with FVG found")
    return None

def find_retest_and_engulfing(bars_5m, breakout_info, orb, today, verbose=True):
    """
    Wait for retest of ORB high/low, then look for engulfing candle
    """
    print(f"\n🔍 STEP 3: Looking for retest and engulfing candle...")
    
    direction = breakout_info['direction']
    breakout_time = breakout_info['breakout_bar']['dt_ny']
    
    print(f"   Direction: {direction}")
    target_level = orb['high'] if direction == 'LONG' else orb['low']
    print(f"   Looking for retest of: {target_level:.2f}")
    
    # Get bars after breakout
    bars_after_breakout = []
    for bar in bars_5m:
        if bar['date'] == today and bar['dt_ny'] > breakout_time:
            bars_after_breakout.append(bar)
    
    bars_after_breakout.sort(key=lambda x: x['dt_ny'])
    
    if verbose:
        print(f"\n   Scanning {len(bars_after_breakout)} bars after breakout...")
    
    # Look for retest
    retest_found = False
    retest_bar_idx = -1
    
    for i, bar in enumerate(bars_after_breakout):
        if verbose:
            print(f"\n   ⏰ {bar['dt_ny'].strftime('%I:%M %p')} - Checking for retest...")
            print(f"      H={bar['h']:.2f} L={bar['l']:.2f} C={bar['c']:.2f}")
        
        if direction == 'LONG':
            # Looking for bar that touches ORB high or FVG low
            touched = bar['l'] <= orb['high']
            
            if verbose:
                print(f"      Bar low ({bar['l']:.2f}) vs ORB high ({orb['high']:.2f})")
            
            if touched:
                print(f"   ✅ RETEST FOUND at {bar['dt_ny'].strftime('%I:%M %p')}")
                print(f"      Low: {bar['l']:.2f} touched/went below {orb['high']:.2f}")
                retest_found = True
                retest_bar_idx = i
                break
            elif verbose:
                print(f"      ❌ No touch yet")
        
        else:  # SHORT
            # Looking for bar that touches ORB low or FVG high
            touched = bar['h'] >= orb['low']
            
            if verbose:
                print(f"      Bar high ({bar['h']:.2f}) vs ORB low ({orb['low']:.2f})")
            
            if touched:
                print(f"   ✅ RETEST FOUND at {bar['dt_ny'].strftime('%I:%M %p')}")
                print(f"      High: {bar['h']:.2f} touched/went above {orb['low']:.2f}")
                retest_found = True
                retest_bar_idx = i
                break
            elif verbose:
                print(f"      ❌ No touch yet")
    
    if not retest_found:
        print(f"\n   ❌ No retest found")
        return None
    
    # Now look for engulfing candle
    retest_bar = bars_after_breakout[retest_bar_idx]
    
    print(f"\n🔍 STEP 4: Looking for engulfing candle after retest...")
    print(f"   Retest bar: H={retest_bar['h']:.2f} L={retest_bar['l']:.2f} C={retest_bar['c']:.2f}")
    
    # Check if next candle(s) engulf the retest candle
    for i in range(retest_bar_idx + 1, min(retest_bar_idx + 5, len(bars_after_breakout))):
        engulf_bar = bars_after_breakout[i]
        
        if verbose:
            print(f"\n   ⏰ {engulf_bar['dt_ny'].strftime('%I:%M %p')} - Checking for engulfing...")
            print(f"      H={engulf_bar['h']:.2f} L={engulf_bar['l']:.2f} C={engulf_bar['c']:.2f}")
        
        if direction == 'LONG':
            # Bullish engulfing: close above retest bar's high
            is_engulfing = engulf_bar['c'] > retest_bar['h']
            
            if verbose:
                print(f"      Engulf close ({engulf_bar['c']:.2f}) vs Retest high ({retest_bar['h']:.2f})")
            
            if is_engulfing:
                print(f"\n   ✅✅✅ BULLISH ENGULFING CANDLE ✅✅✅")
                print(f"   Time: {engulf_bar['dt_ny'].strftime('%I:%M %p')}")
                print(f"   Engulf close: {engulf_bar['c']:.2f} > Retest high: {retest_bar['h']:.2f}")
                print(f"   Entry candle: O={engulf_bar['o']:.2f} H={engulf_bar['h']:.2f} L={engulf_bar['l']:.2f} C={engulf_bar['c']:.2f}")
                
                return {
                    'entry_bar': engulf_bar,
                    'swing_low': retest_bar['l'],
                    'retest_bar': retest_bar
                }
            elif verbose:
                print(f"      ❌ Not engulfing yet")
        
        else:  # SHORT
            # Bearish engulfing: close below retest bar's low
            is_engulfing = engulf_bar['c'] < retest_bar['l']
            
            if verbose:
                print(f"      Engulf close ({engulf_bar['c']:.2f}) vs Retest low ({retest_bar['l']:.2f})")
            
            if is_engulfing:
                print(f"\n   ✅✅✅ BEARISH ENGULFING CANDLE ✅✅✅")
                print(f"   Time: {engulf_bar['dt_ny'].strftime('%I:%M %p')}")
                print(f"   Engulf close: {engulf_bar['c']:.2f} < Retest low: {retest_bar['l']:.2f}")
                print(f"   Entry candle: O={engulf_bar['o']:.2f} H={engulf_bar['h']:.2f} L={engulf_bar['l']:.2f} C={engulf_bar['c']:.2f}")
                
                return {
                    'entry_bar': engulf_bar,
                    'swing_high': retest_bar['h'],
                    'retest_bar': retest_bar
                }
            elif verbose:
                print(f"      ❌ Not engulfing yet")
    
    print(f"\n   ❌ No engulfing candle found after retest")
    return None

def simulate_trade_realistic(direction, entry, stop, target, future_bars):
    """
    Realistic trade simulation with slippage and commissions
    """
    # Apply slippage to entry
    if direction == 'LONG':
        entry_with_slippage = entry + SLIPPAGE_POINTS
        stop_with_slippage = stop - SLIPPAGE_POINTS
        target_with_slippage = target - SLIPPAGE_POINTS
    else:
        entry_with_slippage = entry - SLIPPAGE_POINTS
        stop_with_slippage = stop + SLIPPAGE_POINTS
        target_with_slippage = target + SLIPPAGE_POINTS
    
    for bar in future_bars:
        if direction == 'LONG':
            stop_hit = bar['l'] <= stop_with_slippage
            target_hit = bar['h'] >= target_with_slippage
            
            # If both hit in same bar, check which was closer
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (stop_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (target_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
        
        else:  # SHORT
            stop_hit = bar['h'] >= stop_with_slippage
            target_hit = bar['l'] <= target_with_slippage
            
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (entry_with_slippage - stop_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (entry_with_slippage - target_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
    
    # Trade still open
    if future_bars:
        exit_price = future_bars[-1]['c']
        if direction == 'LONG':
            gross_pnl = (exit_price - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
        else:
            gross_pnl = (entry_with_slippage - exit_price) * POINT_VALUE * POSITION_SIZE
        
        commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
        net_pnl = gross_pnl - commissions
        return {'result': 'OPEN', 'pnl': net_pnl, 'exit_price': exit_price}
    
    return {'result': 'OPEN', 'pnl': 0, 'exit_price': entry_with_slippage}

# ==================== BACKTEST ====================
def backtest_day(bars_15m, bars_5m, target_date, verbose=False):
    """Backtest a single day"""
    
    # Step 1: Get ORB range (9:30-9:45 AM)
    orb = get_orb_range(bars_15m, target_date, verbose=verbose)
    if orb is None:
        return None, "No ORB range found"
    
    # Step 2: Find breakout with FVG
    breakout_info = find_breakout_with_fvg(bars_5m, orb, target_date, verbose=verbose)
    if breakout_info is None:
        return None, "No valid breakout with FVG"
    
    # Step 3: Find retest and engulfing candle
    entry_info = find_retest_and_engulfing(bars_5m, breakout_info, orb, target_date, verbose=verbose)
    if entry_info is None:
        return None, "No valid retest/engulfing pattern"
    
    # Step 4: Calculate trade parameters
    direction = breakout_info['direction']
    entry_price = entry_info['entry_bar']['c']
    entry_time = entry_info['entry_bar']['dt_ny']
    
    if direction == 'LONG':
        stop_loss = entry_info['swing_low']
        risk = entry_price - stop_loss
        take_profit = entry_price + (risk * RISK_REWARD_RATIO)
    else:
        stop_loss = entry_info['swing_high']
        risk = stop_loss - entry_price
        take_profit = entry_price - (risk * RISK_REWARD_RATIO)
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"💡 TRADE SETUP CONFIRMED")
        print(f"{'='*70}")
        print(f"   Direction: {direction}")
        print(f"   Entry: {entry_price:.2f} @ {entry_time.strftime('%I:%M %p')}")
        print(f"   Stop Loss: {stop_loss:.2f}")
        print(f"   Take Profit: {take_profit:.2f}")
        print(f"   Risk: {risk:.2f} points")
        print(f"   Reward: {risk * RISK_REWARD_RATIO:.2f} points (2R)")
    
    # Step 5: Get future bars for simulation
    future_bars = []
    for bar in bars_5m:
        if bar['date'] == target_date and bar['dt_ny'] > entry_time:
            future_bars.append(bar)
    
    future_bars.sort(key=lambda x: x['dt_ny'])
    
    if verbose:
        print(f"\n🔍 Simulating trade with {len(future_bars)} future bars...")
    
    # Step 6: Simulate trade
    outcome = simulate_trade_realistic(direction, entry_price, stop_loss, take_profit, future_bars)
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"TRADE RESULT: {outcome['result']}")
        print(f"{'='*70}")
        print(f"   Exit Price: {outcome['exit_price']:.2f}")
        print(f"   P&L: ${outcome['pnl']:.2f}")
    
    trade = {
        'date': target_date,
        'time': entry_time,
        'type': direction,
        'entry': entry_price,
        'stop': stop_loss,
        'target': take_profit,
        'risk': risk,
        'result': outcome['result'],
        'pnl': outcome['pnl'],
        'exit_price': outcome['exit_price'],
        'orb_high': orb['high'],
        'orb_low': orb['low']
    }
    
    return [trade], orb

# ==================== MAIN ====================
def main():
    print("=" * 70)
    print("OPENING RANGE BREAKOUT STRATEGY - Video Method")
    print("=" * 70)
    print("Rules:")
    print("1. Mark 9:30-9:45 AM range (first 15-min candle)")
    print("2. Wait for 5-min breakout WITH Fair Value Gap overlapping range")
    print("3. Wait for retest of range high/low")
    print("4. Enter on engulfing candle confirmation")
    print("5. Stop: swing low/high | Target: 2R")
    print("=" * 70)
    
    # Get today's date
    ny_tz = pytz.timezone('America/New_York')
    today = datetime.now(ny_tz).date()
    
    print(f"\n[1/3] Authenticating...")
    headers = authenticate()
    print("✅ Authenticated")
    
    print("\n[2/3] Fetching data...")
    print("   Fetching 15-min bars...")
    bars_15m = fetch_bars(headers, unit=2, unit_number=15)
    print(f"   ✅ {len(bars_15m)} bars")
    
    print("   Fetching 5-min bars...")
    bars_5m = fetch_bars(headers, unit=2, unit_number=5)
    print(f"   ✅ {len(bars_5m)} bars")
    
    bars_15m = parse_bars_with_ny_time(bars_15m)
    bars_5m = parse_bars_with_ny_time(bars_5m)
    
    # Get unique dates
    unique_dates = sorted(list(set([bar['date'] for bar in bars_15m])), reverse=True)
    print(f"\n📅 Data available for {len(unique_dates)} days")
    print(f"   Date range: {unique_dates[-1]} to {unique_dates[0]}")
    
    print(f"\n[3/3] Running strategy...")
    print(f"\n🔍 Searching for most recent valid setup (searching quietly)...")
    print(f"   Starting from: {today}")
    
    # Try each day from today backwards
    for test_date in unique_dates:
        print(f"\n   Checking {test_date.strftime('%a %m/%d/%Y')}...", end=" ")
        
        trades, result = backtest_day(bars_15m, bars_5m, test_date, verbose=False)
        
        if trades is not None:
            # Found a valid setup! Now show detailed analysis
            print(f"✅ FOUND!")
            print(f"\n\n{'='*70}")
            print(f"✅ VALID SETUP FOUND ON {test_date.strftime('%A, %B %d, %Y')}")
            print(f"{'='*70}")
            
            # Re-run with verbose to show all details
            print(f"\n🔍 Running detailed analysis...")
            trades, result = backtest_day(bars_15m, bars_5m, test_date, verbose=True)
            
            # Display detailed results
            display_detailed_results(trades, result, test_date)
            return
        else:
            # Show which step failed
            if "ORB" in result:
                print(f"❌ No ORB")
            elif "FVG" in result:
                print(f"❌ No FVG breakout")
            elif "retest" in result:
                print(f"❌ No retest/engulfing")
            else:
                print(f"❌ No setup")
    
    print(f"\n\n{'='*70}")
    print(f"❌ NO VALID SETUPS FOUND")
    print(f"{'='*70}")
    print(f"Searched {len(unique_dates)} days from {unique_dates[-1]} to {unique_dates[0]}")
    print(f"No days met all criteria (ORB + FVG + Retest + Engulfing)")

def display_detailed_results(trades, orb, test_date):
    """Display detailed results with all calculation steps"""
    
    print(f"\n{'='*70}")
    print(f"DETAILED TRADE BREAKDOWN")
    print(f"{'='*70}")
    
    print(f"\n📅 Date: {test_date.strftime('%A, %B %d, %Y')}")
    
    print(f"\n📊 STEP 1: ORB Range (9:30-9:45 AM)")
    print(f"   {'─'*60}")
    print(f"   Open:  {orb.get('open', 'N/A'):.2f}")
    print(f"   High:  {orb['high']:.2f}")
    print(f"   Low:   {orb['low']:.2f}")
    print(f"   Close: {orb.get('close', 'N/A'):.2f}")
    print(f"   Size:  {orb['size']:.2f} points")
    
    for trade in trades:
        print(f"\n{'─'*70}")
        print(f"TRADE EXECUTION")
        print(f"{'─'*70}")
        
        print(f"\n📍 STEP 2: Breakout Direction")
        print(f"   Direction: {trade['type']}")
        print(f"   Entry Time: {trade['time'].strftime('%I:%M %p')}")
        
        print(f"\n💰 STEP 3: Trade Parameters")
        print(f"   Entry Price:  {trade['entry']:.2f}")
        print(f"   Stop Loss:    {trade['stop']:.2f}")
        print(f"   Take Profit:  {trade['target']:.2f}")
        print(f"   Risk:         {trade['risk']:.2f} points")
        print(f"   Reward:       {trade['risk'] * RISK_REWARD_RATIO:.2f} points (2R)")
        print(f"   Risk/Reward:  1:{RISK_REWARD_RATIO}")
        
        print(f"\n📈 STEP 4: Trade Outcome")
        
        if trade['result'] == 'WIN':
            symbol = "✅ WIN"
            color = "🟢"
        elif trade['result'] == 'LOSS':
            symbol = "❌ LOSS"
            color = "🔴"
        else:
            symbol = "⏳ OPEN"
            color = "🟡"
        
        print(f"   Result: {color} {symbol}")
        print(f"   Exit Price: {trade['exit_price']:.2f}")
        
        # Calculate points
        if trade['type'] == 'LONG':
            points = trade['exit_price'] - trade['entry']
        else:
            points = trade['entry'] - trade['exit_price']
        
        print(f"   Points: {points:+.2f}")
        print(f"   P&L: ${trade['pnl']:.2f}")
        
        # Break down P&L
        gross_pnl = trade['pnl'] + (COMMISSION_PER_CONTRACT * POSITION_SIZE * 2)
        print(f"\n💵 P&L Breakdown:")
        print(f"   Gross P&L:     ${gross_pnl:.2f}")
        print(f"   Commissions:   -${COMMISSION_PER_CONTRACT * POSITION_SIZE * 2:.2f} ({POSITION_SIZE} contracts × ${COMMISSION_PER_CONTRACT:.2f} × 2 sides)")
        print(f"   Slippage:      ~${SLIPPAGE_POINTS * POINT_VALUE * POSITION_SIZE * 2:.2f} ({SLIPPAGE_POINTS} pts × 2 sides)")
        print(f"   {'─'*60}")
        print(f"   NET P&L:       ${trade['pnl']:.2f}")
    
    print(f"\n{'='*70}")
    print(f"SUMMARY")
    print(f"{'='*70}")
    print(f"\n📊 Position Details:")
    print(f"   Contracts: {POSITION_SIZE}")
    print(f"   Point Value: ${POINT_VALUE}/point")
    print(f"   Position Size: ${POSITION_SIZE * POINT_VALUE}/point movement")
    
    total_pnl = sum(t['pnl'] for t in trades)
    wins = [t for t in trades if t['result'] == 'WIN']
    losses = [t for t in trades if t['result'] == 'LOSS']
    
    print(f"\n📈 Performance:")
    print(f"   Total Trades: {len(trades)}")
    print(f"   Wins: {len(wins)}")
    print(f"   Losses: {len(losses)}")
    if len(trades) > 0:
        print(f"   Win Rate: {len(wins)/len(trades)*100:.1f}%")
    print(f"   Total P&L: ${total_pnl:.2f}")
    
    print(f"\n{'='*70}")
    print(f"✅ ANALYSIS COMPLETE")
    print(f"{'='*70}")
    print(f"\n💡 TIP: Compare these values with your charting platform:")
    print(f"   1. Verify the ORB range high/low at 9:30-9:45 AM")
    print(f"   2. Check the breakout candle has an FVG")
    print(f"   3. Confirm the retest and engulfing pattern")
    print(f"   4. Validate entry, stop, and target prices")

if __name__ == "__main__":
    main()

OPENING RANGE BREAKOUT STRATEGY - Video Method
Rules:
1. Mark 9:30-9:45 AM range (first 15-min candle)
2. Wait for 5-min breakout WITH Fair Value Gap overlapping range
3. Wait for retest of range high/low
4. Enter on engulfing candle confirmation
5. Stop: swing low/high | Target: 2R

[1/3] Authenticating...
✅ Authenticated

[2/3] Fetching data...
   Fetching 15-min bars...
   ✅ 2009 bars
   Fetching 5-min bars...
   ✅ 6025 bars

📅 Data available for 27 days
   Date range: 2026-01-06 to 2026-02-05

[3/3] Running strategy...

🔍 Searching for most recent valid setup (searching quietly)...
   Starting from: 2026-02-05

   Checking Thu 02/05/2026... ❌ No FVG breakout

   Checking Wed 02/04/2026... 
🔍 STEP 3: Looking for retest and engulfing candle...
   Direction: SHORT
   Looking for retest of: 25230.00
   ✅ RETEST FOUND at 10:05 AM
      High: 25250.75 touched/went above 25230.00

🔍 STEP 4: Looking for engulfing candle after retest...
   Retest bar: H=25250.75 L=25149.00 C=25239.75

   ❌ 